In [1]:
! pip install wandb

In [2]:
! pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/DeepA2')

In [4]:
# Small CNN Model with flexibile  conv blocks
class SmallCNNModel(pl.LightningModule):
    def __init__(self, conv_filters=[32, 32, 32, 32, 32],
                 kernel_size=[3, 3, 3, 3, 3],
                 conv_activation='ReLU',
                 dense_neurons=128,
                 dense_activation='ReLU',
                 dropout=0.0,
                 batch_normalise=False,
                 lr=1e-3):
        super(SmallCNNModel, self).__init__()
        self.save_hyperparameters()
        # Dictionary mapping activation names to function
        activations = {
            'ReLU': nn.ReLU,
            'GELU': nn.GELU,
            'SiLU': nn.SiLU,
            'Mish': nn.Mish

        }
        conv_act = activations.get(conv_activation, nn.ReLU)
        dense_act = activations.get(dense_activation, nn.ReLU)

        self.conv_layers = nn.ModuleList()
        in_channels = 3  # iNature12K images have 3 dimension (RGB)
        for i in range(5):
            out_channels = conv_filters[i]
            kernel_sz = kernel_size[i]
            conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_sz, padding=kernel_sz//2)
            block = [conv]
            if batch_normalise:
                block.append(nn.BatchNorm2d(out_channels))
            block.append(conv_act())
            block.append(nn.MaxPool2d(2))
            self.conv_layers.append(nn.Sequential(*block))
            in_channels = out_channels

        # Compute the flattened feature size after conv layers
        # Assume a default input image size of 224x224
        dummy_input = torch.zeros(1, 3, 224, 224)
        with torch.no_grad():
            features = dummy_input
            for layer in self.conv_layers:
                features = layer(features)
            self.flattened_size = features.view(1, -1).shape[1]

        # Dense layer followed by the output layer
        self.dense = nn.Linear(self.flattened_size, dense_neurons)
        self.dense_activation = dense_act()
        self.dropout = nn.Dropout(dropout) if dropout > 0 else None
        self.out = nn.Linear(dense_neurons, 10)
        self.lr = lr

    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)
        x = torch.flatten(x, 1)
        x = self.dense(x)
        x = self.dense_activation(x)
        if self.dropout:
            x = self.dropout(x)
        x = self.out(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log('test_loss', loss)
        self.log('test_acc', acc)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [5]:
class iNaturalistDataModule(pl.LightningDataModule):
    def __init__(self, data_dir, batch_size=32, num_workers=4, valid_split=0.2, data_augmentation=True):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.valid_split = valid_split
        self.data_augmentation = data_augmentation 

        # Mean and std from ImageNet
        self.mean = [0.485, 0.456, 0.406]
        self.std = [0.229, 0.224, 0.225]

        # Train Transform with Data Augmentation
        if self.data_augmentation:
            self.train_transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.RandomHorizontalFlip(),  # Flip images randomly
                transforms.RandomRotation(10),  # Rotate within ±10 degrees
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust colors
                transforms.ToTensor(),
                transforms.Normalize(self.mean, self.std)  # Normalize
            ])
        else:
            self.train_transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(self.mean, self.std)
            ])

        # No Augmentation
        self.test_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ])

    def setup(self, stage=None):
        # Load dataset
        full_train_dataset = torchvision.datasets.ImageFolder(root=f'{self.data_dir}/train', transform=self.train_transform)
        self.test_dataset = torchvision.datasets.ImageFolder(root=f'{self.data_dir}/val', transform=self.test_transform)

        # **Split train dataset into train 0.8 & validation 0.2
        total = len(full_train_dataset)
        val_size = int(total * self.valid_split)
        train_size = total - val_size
        self.train_dataset, self.val_dataset = random_split(full_train_dataset, [train_size, val_size])

        # **Set validation transform separately**
        self.val_dataset.dataset.transform = self.test_transform
        return self.train_dataset,self.val_dataset,self.test_dataset

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)


In [6]:
# Sweep configuration for wandb hyperparameter search
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_acc',
      'goal': 'maximize'
    },
    'parameters': {
        'conv_filters': {
            'values': [[32, 32, 32, 32, 32], [512, 256, 128, 64, 32], [32, 64, 128, 256, 512]]
        },
        'kernel_size': {
            'values': [[3, 3, 3, 3, 3], [5, 3, 3, 3, 3]]
        },
        'conv_activation': {
            'values': ['ReLU', 'SiLU', 'GeLU','Mish']
        },
        'dense_neurons': {
            'values': [128, 256]
        },
        'dense_activation': {
            'values': ['ReLU', 'SiLU']
        },
        'dropout': {
            'values': [0.2, 0.3]
        },
        'batch_normalise': {
            'values': [True, False]
        },
        'lr': {
            'values': [1e-3, 1e-4]
        },
        'batch_size': {
            'values': [32, 64]
        },
        'data_augmentation': {
            'values': [True, False]
        }
    }
}

In [7]:
def train():
    # Initialize wandb run and get hyperparameters
    wandb.init(project="Small_cnn_model")  # Add project name
    config = wandb.config

    print("Loaded Config:", config)


    model = SmallCNNModel(
        conv_filters=config.get("conv_filters", [32, 32, 32, 32, 32]),  # Default filters
        kernel_size=config.get("kernel_size", [3, 3, 3, 3, 3]),
        conv_activation=config.get("conv_activation", "ReLU"),
        dense_neurons=config.get("dense_neurons", 128),
        dense_activation=config.get("dense_activation", "ReLU"),
        dropout=config.get("dropout", 0.2),
        batch_normalise=config.get("batch_normalise", False),
        lr=config.get("lr", 1e-3)
    )

    data_module = iNaturalistDataModule(
        data_dir='nature_12K',
        batch_size=config.get("batch_size", 64),
        data_augmentation=config.get("data_augmentation", False)
    )
    data_module.setup()
    train_loader = data_module.train_dataloader()
    val_loader = data_module.val_dataloader()
    test_loader = data_module.test_dataloader()

    # Log training process with wandb
    wandb_logger = WandbLogger(project="Small_cnn_model", log_model="all")

    # Define Trainer pytorch_lighting method which automatically trains model and logs metrics
    trainer = pl.Trainer(
        logger=wandb_logger,
        max_epochs=10,
        # accelerator="gpu",
        devices=1,
        precision=16,
        callbacks=[ModelCheckpoint(monitor="val_acc", mode="max")],
        log_every_n_steps=25
    )

    # Train the model
    trainer.fit(model, train_loader,val_loader)





In [ ]:
sweep_id = wandb.sweep(sweep_config,project='Small_cnn_model')

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [ ]:
wandb.agent(sweep_id, train, count=1)

wandb: Agent Starting Run: fmyjfnat with config:
wandb: 	batch_normalise: False
wandb: 	batch_size: 32
wandb: 	conv_activation: SiLU
wandb: 	conv_filters: [512, 256, 128, 64, 32]
wandb: 	data_augmentation: True
wandb: 	dense_activation: SiLU
wandb: 	dense_neurons: 128
wandb: 	dropout: 0.2
wandb: 	kernel_size: [5, 3, 3, 3, 3]
wandb: 	lr: 0.001
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Loaded Config: {'batch_normalise': False, 'batch_size': 32, 'conv_activation': 'SiLU', 'conv_filters': [512, 256, 128, 64, 32], 'data_augmentation': True, 'dense_activation': 'SiLU', 'dense_neurons': 128, 'dropout': 0.2, 'kernel_size': [5, 3, 3, 3, 3], 'lr': 0.001}


c:\ProgramData\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:513: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\ProgramData\anaconda3\Lib\site-packages\pytorch_lightning\loggers\wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

  | Name             | Type       | Params | Mode 
--------------------------------------------------------
0 | conv_layers      | ModuleList | 1.6 M  | train
1 | dense            | Linear     | 200 K  | train
2 | dense_activation | SiLU       | 0      | train
3 | dropout          | Dropout    | 0      | train
4 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\ProgramData\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
c:\ProgramData\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

In [10]:
wandb.agent(sweep_id, train, count=30)  # Runs 30 experiments

wandb: Agent Starting Run: 2mvizmp0 with config:
wandb: 	batch_normalise: False
wandb: 	batch_size: 32
wandb: 	conv_activation: GeLU
wandb: 	conv_filters: [32, 32, 32, 32, 32]
wandb: 	data_augmentation: False
wandb: 	dense_activation: SiLU
wandb: 	dense_neurons: 128
wandb: 	dropout: 0.3
wandb: 	kernel_size: [3, 3, 3, 3, 3]
wandb: 	lr: 0.0001


Loaded Config: {'batch_normalise': False, 'batch_size': 32, 'conv_activation': 'GeLU', 'conv_filters': [32, 32, 32, 32, 32], 'data_augmentation': False, 'dense_activation': 'SiLU', 'dense_neurons': 128, 'dropout': 0.3, 'kernel_size': [3, 3, 3, 3, 3], 'lr': 0.0001}


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name             | Type       | Params | Mode 
--------------------------------------------------------
0 | conv_layers      | ModuleList | 37.9 K | train
1 | dense            | Linear     | 200 K  | train
2 | dense_activation | SiLU       | 0      | train
3 | dropout          | Dropout    | 0      | train
4 | out              | Linear     | 1.3 K  | train
--------------------------------------------------------
240 K     Trainable params
0         Non-trainable params
240 K     Total params
0.960     Total estimated model para

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

wandb: ERROR Run bf1nm0c2 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-6-12ba6624b3f8>", line 46, in train
wandb: ERROR     trainer.fit(model, train_loader,val_loader)
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 561, in fit
wandb: ERROR     call._call_and_handle_interrupt(
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/call.py", line 48, in _call_and_handle_interrupt
wandb: ERROR     return trainer_fn(*args, **kwargs)
wandb: ERROR            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 599, in _fit_impl
wandb: ERROR     self._run(model, ckpt_path=ckpt_path)
wandb: ERROR   File "/usr/local/lib/python3.11/dist-pack

Loaded Config: {'batch_normalise': False, 'batch_size': 32, 'conv_activation': 'SiLU', 'conv_filters': [512, 256, 128, 64, 32], 'data_augmentation': False, 'dense_activation': 'SiLU', 'dense_neurons': 128, 'dropout': 0.3, 'kernel_size': [5, 3, 3, 3, 3], 'lr': 0.0001}


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name             | Type       | Params | Mode 
--------------------------------------------------------
0 | conv_layers      | ModuleList | 1.6 M  | train
1 | dense            | Linear     | 200 K  | train
2 | dense_activation | SiLU       | 0      | train
3 | dropout          | Dropout    | 0      | train
4 | out              | Linear     | 1.3 K  | train
--------------------------------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params
7.233     Total estimated model para

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

wandb: ERROR Run mi0k9che errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-6-12ba6624b3f8>", line 46, in train
wandb: ERROR     trainer.fit(model, train_loader,val_loader)
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 561, in fit
wandb: ERROR     call._call_and_handle_interrupt(
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/call.py", line 48, in _call_and_handle_interrupt
wandb: ERROR     return trainer_fn(*args, **kwargs)
wandb: ERROR            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/trainer.py", line 599, in _fit_impl
wandb: ERROR     self._run(model, ckpt_path=ckpt_path)
wandb: ERROR   File "/usr/local/lib/python3.11/dist-pack

In [ ]:
wandb.agent(sweep_id, train, count=1)

# Part A Question:4

In [ ]:
# best model from training :'batch_size': 32, 'conv_activation': 'SiLU', 'conv_filters': [512, 256, 128, 64, 32], 'kernel_size': [3, 3, 3, 3, 3], 'dense_activation': 'SiLU', 'dense_neurons': 128, 'dropout': 0.3, 'lr': 0.0001, 'data_augmentation': False,

In [10]:

import torch
import wandb
import pytorch_lightning as pl
import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from kaggle_secrets import UserSecretsClient
from torchvision.utils import make_grid

# load WandB API Key from Kaggle Secrets
# user_secrets = UserSecretsClient()
# wandb_key = user_secrets.get_secret("wandb")
wandb.login(key='2dc5ef867eb84d722a7aa9a5f6b133f70c361613')

#  Initialize WandB Project
wandb.init(project="Small_cnn_model", name="best_model_eval_")

torch.cuda.empty_cache() # if gpu has not enough space

#  Load the best model from sweeps
best_model = SmallCNNModel(
    # best model from training :'batch_size': 32, 'conv_activation': 'SiLU', 'conv_filters': [512, 256, 128, 64, 32], 'kernel_size': [3, 3, 3, 3, 3], 'dense_activation': 'SiLU', 'dense_neurons': 128, 'dropout': 0.3, 'lr': 0.0001, 'data_augmentation': False,
    conv_filters=[512, 256, 128, 64, 32],
    kernel_size=[3, 3, 3, 3, 3],
    conv_activation="SiLU",
    dense_neurons=128,
    dense_activation="SiLU",
    dropout=0.3,
    batch_normalise=True,
    lr=0.0001
)

#  Load dataset
data_module = iNaturalistDataModule(
    data_dir='/kaggle/input/nature12k/inaturalist_12K',
    batch_size=32,
    data_augmentation=False
)
data_module.setup()

test_loader = data_module.test_dataloader()

# Load WandB Logger
wandb_logger = WandbLogger(project="Small_cnn_model", log_model="all")


trainer = pl.Trainer(
    max_epochs=10,
    logger=wandb_logger,
    accelerator="gpu"    
)

trainer.fit(best_model, data_module)

test_results = trainer.test(best_model, test_loader)
test_accuracy = test_results[0]['test_acc']


wandb.log({"Test Accuracy": test_accuracy})



def log_test_predictions(model, dataloader, num_classes=10, num_per_class=3):
    model.eval()
    class_images = {i: [] for i in range(num_classes)}  # Store images per class

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            # Collect 3 images per class
            for img, label, pred in zip(images, labels, preds):
                if len(class_images[label.item()]) < num_per_class:
                    class_images[label.item()].append((img, pred.item()))

            # Stop if we have enough samples
            if all(len(class_images[i]) == num_per_class for i in range(num_classes)):
                break

    # Create a grid
    fig, axes = plt.subplots(num_classes, num_per_class, figsize=(num_per_class * 3, num_classes * 3))

    for class_idx, ax_row in enumerate(axes):
        for img_idx, ax in enumerate(ax_row):
            if class_idx in class_images and len(class_images[class_idx]) > img_idx:
                img, pred_label = class_images[class_idx][img_idx]
                img = img.permute(1, 2, 0).cpu().numpy()

                ax.imshow(img)
                ax.set_title(f"Pred: {pred_label}", fontsize=10)
                ax.axis("off")

    plt.tight_layout()

    # Log to WandB
    wandb.log({"Test Predictions": wandb.Image(fig, caption="Per-Class Predictions (10x3)")})
    plt.close(fig)


test_dataloader = data_module.test_dataloader()
log_test_predictions(best_model, test_dataloader)

wandb.finish()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.06 GiB. GPU 0 has a total capacity of 14.74 GiB of which 758.12 MiB is free. Process 3076 has 14.00 GiB memory in use. Of the allocated memory 13.84 GiB is allocated by PyTorch, and 21.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Questions from PART B

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import wandb
from pytorch_lightning.loggers import WandbLogger
from torchvision.datasets import ImageFolder

# Sweep Configuration
sweep_config = {
    "method": "grid",
    "metric": {"name": "val_acc", "goal": "maximize"},
    "parameters": {
        "epochs": {"values": [5]},
        "batch_size": {"values": [64, 128]},
        "denselayer_size": {"values": [64, 128]},
        "l_rate": {"values": [0.001, 0.0001]},
        "optimizer": {"values": ["Adam"]},
        "dropout": {"values": [0.2, 0.4]},
        "model_version": {"values": ["resnet50"]},
        "activation": {"values": ["relu", "leakyrelu"]}
    }
}

# Create Sweep
sweep_id = wandb.sweep(sweep_config, project="iNature_12K_by_pretrained_Resnet50-Model")

# Define Data Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class FineTuneModel(pl.LightningModule):
    def __init__(self, num_classes=10, model_version="resnet50", denselayer_size=128, dropout=0.4, l_rate=0.001, activation="relu"):
        super(FineTuneModel, self).__init__()
        self.learning_rate = l_rate
        self.activation_fn = nn.ReLU() if activation == "relu" else nn.LeakyReLU()

        self.model = models.__dict__[model_version](pretrained=True)

        # Freeze all layers initially
        for param in self.model.parameters():
            param.requires_grad = False

        # Modify the final classification layer
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Linear(num_ftrs, denselayer_size),
            self.activation_fn,
            nn.Dropout(dropout),
            nn.Linear(denselayer_size, num_classes)
        )

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        loss = self.criterion(outputs, y)
        acc = (outputs.argmax(dim=1) == y).float().mean()
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        loss = self.criterion(outputs, y)
        acc = (outputs.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        outputs = self(x)
        loss = self.criterion(outputs, y)
        acc = (outputs.argmax(dim=1) == y).float().mean()
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer

# Training Function for WandB Sweeps
def train():
    wandb.init()

    # Fetch hyperparameters
    config = wandb.config
    batch_size = config.batch_size
    epochs = config.epochs
    model_version = config.model_version
    denselayer_size = config.denselayer_size
    dropout = config.dropout
    l_rate = config.l_rate
    activation = config.activation

    # Load Data
    data_module = iNaturalistDataModule(
        data_dir='/kaggle/input/nature12k/inaturalist_12K',
        batch_size=batch_size
    )
    data_module.setup()
    train_loader = data_module.train_dataloader()
    val_loader = data_module.val_dataloader()
    test_loader = data_module.test_dataloader()

    # Initialize Model
    model = FineTuneModel(num_classes=10, model_version=model_version, denselayer_size=denselayer_size, dropout=dropout,
                          l_rate=l_rate, activation=activation)

    # Set up WandB Logger
    wandb_logger = WandbLogger(project="iNature_12K_by_pretrained_Resnet50-Model")

    # Train Model
    trainer = pl.Trainer(
        max_epochs=epochs,
        logger=wandb_logger,
        accelerator="gpu" if torch.cuda.is_available() else "cpu"
    )
    trainer.fit(model, train_loader, val_loader)

    # Test Best Model
    trainer.test(model, test_loader)

# Run the Sweep Agent
wandb.agent(sweep_id, function=train, count=20)

Create sweep with ID: bwxazi3u
Sweep URL: https://wandb.ai/ma23c011-indian-institute-of-technology-madras/iNature_12K_by_pretrained_Resnet50-Model/sweeps/bwxazi3u


wandb: Agent Starting Run: 2ugfo6fl with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	denselayer_size: 64
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	l_rate: 0.001
wandb: 	model_version: resnet50
wandb: 	optimizer: Adam


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

epoch,▁▁▂▂▄▄▅▅▅▇▇█
test_acc,▁
test_loss,▁
train_acc,▁▄█▅█▇
train_loss,█▅▃▃▁▂
trainer/global_step,▁▁▂▃▄▅▅▆▆███
val_acc,▁▆▇▇█
val_loss,█▅▃▂▁
epoch,5
test_acc,0.7275
test_loss,1.03252


wandb: Agent Starting Run: dyfokb2d with config:
wandb: 	activation: relu
wandb: 	batch_size: 128
wandb: 	denselayer_size: 128
wandb: 	dropout: 0.2
wandb: 	epochs: 5
wandb: 	l_rate: 0.001
wandb: 	model_version: resnet50
wandb: 	optimizer: Adam


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]